In [ ]:
!pip install pydub
from pydub import AudioSegment
from google.colab import files
import struct

In [ ]:
# Function to convert text into bits
def text_to_bits(text):
    bits = []
    for char in text:
        binary = bin(ord(char))[2:].zfill(8)
        bits.extend([int(bit) for bit in binary])
    return bits

In [ ]:
# Function to convert bits into text
def bits_to_text(bits):
    text = ""
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        char = chr(int("".join([str(bit) for bit in byte]), 2))
        text += char
    return text

In [ ]:
# Function to hide text in an audio file
def hide_text_in_audio(audio_path, output_path, text):
    audio = AudioSegment.from_file(audio_path, format="mp3")

    # Convert the message text into bits
    message_bits = text_to_bits(text)

    # Embed the message length at the beginning of the audio file
    message_length = len(message_bits)
    message_length_bytes = struct.pack(">I", message_length)
    audio_data = bytes(audio.raw_data[:4]) + message_length_bytes + bytes(audio.raw_data[8:])

    # Embed message bits into the audio samples
    for i in range(len(message_bits)):
        sample_index = int(len(audio_data) * (i / len(message_bits)))
        sample = audio_data[sample_index]
        sample = (sample & 0xFE) | message_bits[i]
        audio_data = audio_data[:sample_index] + bytes([sample]) + audio_data[sample_index + 1:]

    # Save the audio file with the embedded message
    with open(output_path, "wb") as f:
        f.write(audio_data)

In [ ]:
# Function to download the output audio file
def download_output_audio(output_path):
    files.download(output_path)

In [ ]:
# Upload the audio file from the computer
uploaded = files.upload()

In [ ]:
# Get the path of the uploaded audio file
audio_path = next(iter(uploaded))

In [ ]:
# Path to save the output audio file with the embedded message
output_path = "/content/output_audio.mp3"

In [ ]:
# Hide the message in the audio file
text = "This is Secret Message"
hide_text_in_audio(audio_path, output_path, text)

In [ ]:
# Download the output audio file with the embedded message
download_output_audio(output_path)